# Machine Learning Model Deployment on OCI Data Science

## 📌 Overview
This notebook demonstrates the **end-to-end workflow** of training, registering, and deploying a machine learning model using **OCI Data Science** services. The goal is to showcase how to **train a model**, add it to the **Model Catalog**, and **deploy it as an API endpoint** for real-time predictions.

## 📌 Workflow Steps
1. **Load and Preprocess Data**  
   - Using the **Iris dataset**, a standard dataset for classification tasks.
   - Splitting the data into **training and testing sets**.

2. **Train a Machine Learning Model**  
   - Training a **Random Forest Classifier** using `scikit-learn`.  
   - Evaluating the model's performance.

3. **Save and Register the Model**  
   - Storing the trained model as a **pickle (`.pkl`) file**.  
   - Uploading the model to the **OCI Model Catalog** for versioning and tracking.

4. **Deploy the Model**  
   - Creating a **Model Deployment** in OCI.  
   - Deploying the model to an **OCI endpoint** for real-time inference.

5. **Test the Deployment**  
   - Sending a test request to the deployed model using an **HTTP request**.
   - Validating the model's response.

## 📌 Why This Matters?
This workflow is **crucial** for building real-world AI applications where models need to be accessible as APIs for **scalable** and **real-time** predictions.

## 📌 Prerequisites
Before running this notebook, ensure:
✅ You have an **OCI Data Science environment** set up.  
✅ Your **authentication** method (Resource Principal / API Key) is configured.  
✅ You have the necessary **permissions** to access OCI Data Science services.

### Libraries

In [1]:
import ads
import os
import tempfile
from ads.catalog.model import ModelCatalog
from ads.common.model import ADSModel
from ads.model.framework.sklearn_model import SklearnModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from ads.model.deployment import ModelDeployer, ModelDeploymentProperties
from ads.common.model_metadata import UseCaseType  

  from ads.common.model_metadata import UseCaseType



In [2]:
import ads

# Set authentication
ads.set_auth("resource_principal")

# Check if authentication is set properly
print(ads.auth.default_signer())

{'config': {'additional_user_agent': 'Oracle-ads/version=2.11.19#surface=DATASCIENCE_NOTEBOOK#api=UNKNOWN'}, 'signer': <oci.auth.signers.ephemeral_resource_principals_signer.EphemeralResourcePrincipalSigner object at 0x7f2c2bd97a50>, 'client_kwargs': {}}


In [3]:
!oci os ns get

{
  "data": "fro8fl9kuqli"
}


## 1. Load dataset and train the model

In [4]:
# Load dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

In [5]:
# Train model
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier()

## 2. Save model in Model Catalog

In [6]:
# Save the trained model
sk_model = SklearnModel(estimator=model, artifact_dir="./artifact")
sk_model.prepare(
    inference_conda_env="automlx242_p38_cpu_x86_64_v1",  # Match the Conda env from your screenshot
    inference_python_version="3.8",  # Explicit Python version
    training_script_path=None,
    force_overwrite=True  # ✅ Add this to overwrite runtime.yaml
)


  warnings.warn("slug will be deprecated. Provide conda pack path instead.")



algorithm: RandomForestClassifier
artifact_dir:
  /home/datascience/1-Customers/Workshops/3-Renesas/artifact:
  - - model.joblib
    - score.py
    - .model-ignore
    - runtime.yaml
    - test_json_output.json
framework: scikit-learn
model_deployment_id: null
model_id: null

In [7]:

# Save the trained model with overwrite
sk_model.save(
    name="iris_model",
    overwrite=True,
    force_overwrite=True,
    metadata={"custom_metadata": "Random Forest trained on Iris dataset"}  # ✅ Ensure values are strings
)


Start loading model.joblib from model directory /home/datascience/1-Customers/Workshops/3-Renesas/artifact ...
Model is successfully loaded.
['model.joblib', 'score.py', '.model-ignore', 'runtime.yaml', 'test_json_output.json']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

'ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaaeicj2tiavlw2o737zvycvt465gmkvqnhpjx4tv6by2m6do44wfna'

#### Debug

In [8]:
import os


# Ensure the environment variable is set correctly
compartment_id = os.environ.get('NB_SESSION_COMPARTMENT_OCID')

if not compartment_id:
    raise ValueError("Environment variable 'NB_SESSION_COMPARTMENT_OCID' is not set.")


print(compartment_id)

ocid1.compartment.oc1..aaaaaaaaknvoqtskvdvzf7ayl7trpo7swj3zgnrwljialccigwkeqymhixyq


In [ ]:
import ads
print(ads.__version__)


In [11]:
print(os.environ['PROJECT_OCID']) # Your OCI project OCID
print(os.environ['NB_SESSION_COMPARTMENT_OCID'])

ocid1.datascienceproject.oc1.eu-frankfurt-1.amaaaaaaeicj2tia3fe52eyzjvk5dra3hawvdz5cuivcdc4i7rp222xtkgiq
ocid1.compartment.oc1..aaaaaaaaknvoqtskvdvzf7ayl7trpo7swj3zgnrwljialccigwkeqymhixyq


## 3. Deploy the trained model
#### Deploying a Model on OCI

This cell **deploys a trained ML model** as an API on OCI.

#### 🔹 Key Steps:
- Set **Compartment, Project, and Model IDs**.
- Configure **compute resources** (CPU, memory, bandwidth).
- Deploy the model using `ModelDeployer()`.
- Print the **deployment ID and endpoint URL**.

✅ **Result:** The model is now live and ready for predictions!


In [10]:
# ✅ Ensure you replace these OCIDs with your actual values
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaaknvoqtskvdvzf7ayl7trpo7swj3zgnrwljialccigwkeqymhixyq"  # Replace with your Compartment OCID
PROJECT_ID = "ocid1.datascienceproject.oc1.eu-frankfurt-1.amaaaaaaeicj2tia3fe52eyzjvk5dra3hawvdz5cuivcdc4i7rp222xtkgiq"  # Replace with your Data Science Project OCID
MODEL_ID = "ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaaeicj2tiaowyd5geqx6wfklutti7blspo3rfthmasdrrf66zsb26q"  # Replace with your saved model's OCID

# ✅ Define the deployment configuration
deployment_properties = (
    ModelDeploymentProperties(MODEL_ID)
    .with_prop("display_name", "Sklearn Model Deployment - Iris Dataset")
    .with_prop("project_id", PROJECT_ID)
    .with_prop("compartment_id", COMPARTMENT_ID)
    .with_instance_configuration(
        config={
            "INSTANCE_SHAPE": "VM.Standard.E4.Flex",  # Ensure the shape is available
            "INSTANCE_COUNT": 1,
            "bandwidth_mbps": 10,
            "memory_in_gbs": 10,
            "ocpus": 2,
        }
    )
)


# ✅ Deploy the model
deployer = ModelDeployer()
deployment_info = deployer.deploy(deployment_properties)

# ✅ Print deployment details
print(f"Deployment OCID: {deployment_info.model_deployment_id}")
print(f"Deployment URL: {deployment_info.url}")



  warnings.warn(

  warnings.warn(

Model Deployment OCID: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiajp23i647sf6aiaa5v7jd3gqxjzf4ndm6bbfglyzb6jva


Creating model deployment:   0%|          | [00:00<?, ?it/s]

Deployment OCID: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiajp23i647sf6aiaa5v7jd3gqxjzf4ndm6bbfglyzb6jva
Deployment URL: https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiajp23i647sf6aiaa5v7jd3gqxjzf4ndm6bbfglyzb6jva


In [ ]:
# Replace with your model deployment OCID
model_deployment_id = "ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiaknnybr7kow3qvy3ybkdrnayvz5vgnbalombbymmqkzla"

# Check deployment status
deployment = ModelDeployment.from_id(model_deployment_id)
print("Deployment Status:", deployment.status)

## 4. Test the deployed model

### 4.1. Troubleshooting in case of "NotAuthorizedOrNotFound" Error

#### Set resource principal authentication

In [ ]:
import ads

# Set authentication
ads.set_auth("resource_principal")

# Check if authentication is set properly
print(ads.auth.default_signer())


#### List all environment variables

In [13]:
import os
print(os.environ.keys())  # List all environment variables

KeysView(environ({'JPY_SESSION_NAME': '/home/datascience/1-Customers/Workshops/3-Renesas/model_deployment.ipynb', 'OCI_RESOURCE_PRINCIPAL_VERSION': '2.2', 'CUDNN_VERSION': '7.6.5.32', 'CONDA_SHLVL': '2', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib', 'CONDA_EXE': '/opt/conda/bin/conda', 'REQUESTS_CA_BUNDLE': '/etc/pki/ca-trust/extracted/pem/tls-ca-bundle.pem', 'DATASCIENCE_UID': '1000', 'OCI_RESOURCE_PRINCIPAL_PRIVATE_PEM': '/etc/ssl/rp_key.pem', 'LANG': 'en_US.UTF-8', 'HISTCONTROL': 'ignoredups', 'HOSTNAME': '', 'JUPYTER_SERVER_LOG_FILE': 'jupyterlab.log', 'KMP_INIT_AT_FORK': 'FALSE', 'TENANCY_OCID': 'ocid1.tenancy.oc1..aaaaaaaajlqw7qs62agvha73f25ivmhatqin2fmrdppma2mh4kf7pjqe6vqa', 'NB_SESSION_COMPARTMENT_OCID': 'ocid1.compartment.oc1..aaaaaaaaknvoqtskvdvzf7ayl7trpo7swj3zgnrwljialccigwkeqymhixyq', 'OCI_INTERNAL_RUNTIME_CONFIG_FILE_PATH': '/etc/nb-runtime-config', 'CONDA_PREFIX': '/home/datascience/conda/generalml_p311_cpu_x86_64_v1', 'NVIDIA_VISIBLE

In [14]:
compartment_id = os.environ.get("NB_SESSION_COMPARTMENT_OCID")

if compartment_id:
    print(f"✅ Compartment ID: {compartment_id}")
else:
    print("❌ ERROR: Compartment ID not found!")


✅ Compartment ID: ocid1.compartment.oc1..aaaaaaaaknvoqtskvdvzf7ayl7trpo7swj3zgnrwljialccigwkeqymhixyq


#### Check if you have access to any projects

In [ ]:
from oci.data_science import DataScienceClient
import ads

ads.set_auth("resource_principal")

# Get Compartment ID
compartment_id = os.getenv("NB_SESSION_COMPARTMENT_OCID")

# Initialize OCI Data Science Client
ds_client = DataScienceClient(config={}, signer=ads.auth.default_signer()["signer"])

try:
    projects = ds_client.list_projects(compartment_id=compartment_id)
    print("✅ List of Projects:", projects.data)
except Exception as e:
    print("❌ ERROR:", e)


### 4.2. Get your model deployment ID

In [16]:
from oci.data_science import DataScienceClient
import ads

# Set authentication
ads.set_auth("resource_principal")

# Get compartment ID
compartment_id = os.getenv("NB_SESSION_COMPARTMENT_OCID")

# Initialize Data Science Client
ds_client = DataScienceClient(config={}, signer=ads.auth.default_signer()["signer"])

# List all model deployments
deployments = ds_client.list_model_deployments(compartment_id=compartment_id)

# Print all available deployments
for deployment in deployments.data:
    print(f"Deployment Name: {deployment.display_name}, OCID: {deployment.id}")


Deployment Name: Sklearn Model Deployment, OCID: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiaknnybr7kow3qvy3ybkdrnayvz5vgnbalombbymmqkzla
Deployment Name: datasciencemodeldeployment20250228123859, OCID: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiail3fiiqylyrsykl7hvfdpcb2uuudqyoldhyi7eo2fbba
Deployment Name: skydance-model-deployment, OCID: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tian3ruu63f6o5ytx742dpbxeauynkifglxj5hwhh2whgbq


### 3.3. Get Deployment endpoint

In [17]:
deployment_id = "ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiaknnybr7kow3qvy3ybkdrnayvz5vgnbalombbymmqkzla"

# Get model deployment details
deployment_details = ds_client.get_model_deployment(deployment_id)
deployment_url = deployment_details.data.model_deployment_url

print(f"✅ Model Deployment URL: {deployment_url}")


✅ Model Deployment URL: https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiaknnybr7kow3qvy3ybkdrnayvz5vgnbalombbymmqkzla


### 3.4. Send a test prediction request

In [8]:
print(deployment_details.data.lifecycle_state)


ACTIVE


In [9]:
import requests
import oci
from oci.auth.signers import get_resource_principals_signer
from oci.config import from_file
from oci.signer import Signer
import json

# ✅ **Set Your Model Deployment URL**
endpoint = "https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeicj2tiaknnybr7kow3qvy3ybkdrnayvz5vgnbalombbymmqkzla"  # Example: https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.xxxxx

# ✅ **Define Input Data (Update as Needed)**
'''input_data = {
    #"input": [1.0, 2.0, 3.0, 3.0]
    "input": [[5.1, 3.5, 1.4, 0.2]] # Adjust this according to your model's expected format
}'''
input_data = [[5.1, 3.5, 1.4, 0.2]]

# ✅ **Choose Authentication Method**
use_rps = True  # Set to False if using API key authentication

if use_rps:
    print("🔹 Using Resource Principal Authentication")
    auth = get_resource_principals_signer()
else:
    print("🔹 Using Config & API Key Authentication")
    config = from_file("~/.oci/config")  # Replace with your actual config path
    auth = Signer(
        tenancy=config['tenancy'],
        user=config['user'],
        fingerprint=config['fingerprint'],
        private_key_file_location=config['key_file'],
        pass_phrase=config.get('pass_phrase')
    )

# ✅ **Set Headers**
headers = {
    "Content-Type": "application/json"
}

# ✅ **Make the API Call**
try:
    response = requests.post(endpoint + "/predict", headers=headers, json=input_data, auth=auth)

    if response.status_code == 200:
        print("✅ Prediction Success:", response.json())
    else:
        print(f"❌ Error {response.status_code}: {response.text}")

except Exception as e:
    print(f"🚨 Exception Occurred: {e}")


🔹 Using Resource Principal Authentication
✅ Prediction Success: {'prediction': [0]}
